# 分析不合规合同


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path


In [2]:
pd.options.display.max_rows = 5


In [3]:
data_dir_name = 'data'
data_dir = Path.cwd() / data_dir_name
if not data_dir.exists():
    data_dir.mkdir()
data_dir


PosixPath('/Users/levin/workspace/git-repositories/anaconda/study-pandas-tutorials/Work/data')

In [4]:
lp = '2022-03-18'
tp = '2022-03-25'
# lp: last period
lp_filename = f'non-compliant_contracts-{lp}.xlsx'
lp_sheet_name = 'Sheet1'

# tp: this period
tp_filename = f'non-compliant_contracts-{tp}.xlsx'
tp_sheet_name = 'Sheet1'

# common
key_fields = ['合同编号']


## 读取上期数据


In [5]:
lp_path = data_dir / lp_filename
lp_path


PosixPath('/Users/levin/workspace/git-repositories/anaconda/study-pandas-tutorials/Work/data/non-compliant_contracts-2022-03-18.xlsx')

In [6]:
lp_db = pd.read_excel(lp_path, lp_sheet_name)
# deduplication based on 'contract number' field
lp_db = lp_db.drop_duplicates(subset=key_fields)
lp_db


,序号,分公司,项目部,organ_id,项目名称,资源名称,合同编号,甲方名称,乙方名称,情况,说明
0,1,产园-深圳公司,蛇口网谷,1032,科健大厦,科健大厦-广告位2,kjds-2022-02-0109,深圳市招商创业有限公司,驰众广告有限公司,倒签,已审批
1,2,产园-重庆公司,金山意库,1412212,金山意库,9号楼-场地租赁,jsyk-2022-02-0660,重庆招商金山意库商业管理有限公司,丁思明,倒签,已审批
...,...,...,...,...,...,...,...,...,...,...,...
466,467,产园-深圳公司,蛇口网谷,1431202,万融大厦,万融大厦C座7层-702,wrds-2021-03-1031,深圳市万融大厦管理有限公司,深圳市敢为软件技术有限公司,应结未结,已终止未结算
471,472,园区运营中心,南海意库-商业,1433221,南海意库-商业,6栋1层-110,nhyk-sy-2021-12-1131,招商局蛇口工业区控股股份有限公司,深圳剪刀侠美发管理有限公司,应结未结,已终止未结算


## 读取本期数据


In [7]:
tp_path = data_dir / tp_filename
tp_path


PosixPath('/Users/levin/workspace/git-repositories/anaconda/study-pandas-tutorials/Work/data/non-compliant_contracts-2022-03-25.xlsx')

In [8]:
tp_db = pd.read_excel(tp_path, tp_sheet_name)
tp_db = tp_db.drop_duplicates(subset=key_fields)
tp_db


,序号,分公司,项目部,organ_id,Unnamed: 4,项目名称,资源名称,合同编号,甲方名称,乙方名称,情况,说明
0,1,产园-深圳公司,蛇口网谷,1032,1008,科健大厦,科健大厦-广告位2,kjds-2022-02-0109,深圳市招商创业有限公司,驰众广告有限公司,倒签,已审批
1,2,产园-重庆公司,金山意库,1412212,1410209,金山意库,9号楼-场地租赁,jsyk-2022-02-0660,重庆招商金山意库商业管理有限公司,丁思明,倒签,已审批
...,...,...,...,...,...,...,...,...,...,...,...,...
425,426,产园-深圳公司,蛇口网谷,1430201,1431203,万海大厦,"万海大厦C座4层-401,万海大厦C座4层-402",WHDXX-2019-03-0228,深圳市万海大厦管理有限公司,深圳小虫时尚设计有限公司,应结未结,已终止未结算
429,430,园区运营中心,南海意库-商业,1433221,1433220,南海意库-商业,6栋1层-110,nhyk-sy-2021-12-1131,招商局蛇口工业区控股股份有限公司,深圳剪刀侠美发管理有限公司,应结未结,已终止未结算


## 提取增量数据

从本期数据中提取本期增量数据, 为了分析本期增量情况.

使用 `pandas` 做这种操作稍微有些复杂, 基本思路如下:

1. 使用 `left join` 模式将本期数据与上期数据进行 `merge`, 本期数据作为 `left`
2. 在使用 `merge` 函数时, 需要设置 `indicator` 参数为 `True`. 这样就可以在 `merge` 的结果表中增加一列用于表明 `merge` 的方式
3. 利用 `indicator` 来创建一个 Boolean indexing, 从而可以提取出增量数据


In [9]:
db_all = tp_db.merge(
    lp_db,
    on=key_fields,
    how='left',
    indicator=True,
    suffixes=(None, '_y')
)
incremental_flag = db_all['_merge'] == 'left_only'
incremental_db = db_all[tp_db.columns][incremental_flag]
incremental_db


,序号,分公司,项目部,organ_id,Unnamed: 4,项目名称,资源名称,合同编号,甲方名称,乙方名称,情况,说明
28,29,产园-深圳公司,光明科技园,1413273,1412251,招商局光明科技园,"宿舍B7栋宿舍2层-B7-205,宿舍B7栋宿舍3层-B7-322,宿舍B7栋宿舍5层-B7...",招光加22C016,招商局光明科技园有限公司,深圳市活水床旁诊断仪器有限公司,倒签,已审批
45,46,产园-深圳公司,光明科技园,1413273,1412251,招商局光明科技园,公寓B10栋18层-B10-1803,招光加22C022,招商局光明科技园有限公司,阳芬,倒签,已审批
...,...,...,...,...,...,...,...,...,...,...,...,...
332,371,产园-深圳公司,光明科技园,1413273,1412251,招商局光明科技园,公寓B10栋18层-B10-1803,招光加21C073,招商局光明科技园有限公司,阳芬,应结未结,已终止未结算
352,426,产园-深圳公司,蛇口网谷,1430201,1431203,万海大厦,"万海大厦C座4层-401,万海大厦C座4层-402",WHDXX-2019-03-0228,深圳市万海大厦管理有限公司,深圳小虫时尚设计有限公司,应结未结,已终止未结算


## 分析本期数据

分析思路:

1. 根据 `['分公司', '项目部', '情况']` 分组, 计算数量
2. 分组小计
3. 按分组小计倒排序


In [10]:
def analyze(_df):
    tp_grp = _df.groupby(['分公司', '项目部', '情况'])
    tp_grp_unstacked = tp_grp['organ_id'].count().unstack([-1, -2])
    subtotal = '--小计--'
    tp_grp_unstacked[('', subtotal)] = \
        tp_grp_unstacked.apply(lambda s: s.sum(), axis=1)

    tp_grp_stacked = tp_grp_unstacked.sort_values(
        ('', subtotal), ascending=False).stack([-1, -2]).to_frame('数量')
    tp_grp_stacked.loc[('总计', '', '')] = tp_grp['organ_id'].count().sum()

    return tp_grp_stacked.astype('int32')


In [11]:
analyze(tp_db)


数量
分公司     项目部    情况       
产园-深圳公司 --小计--       129
        光明科技园  倒签     30
...                  ...
产园-青岛公司 蓝湾网谷   应结未结    2
总计                   354

[48 rows x 1 columns]

## 分析增量数据


In [12]:
analyze(incremental_db)


数量
分公司     项目部     情况      
产园-深圳公司 --小计--        17
        光明科技园   倒签     4
...                   ..
园区运营中心  南海意库-商业 应结未结   1
总计                    35

[18 rows x 1 columns]

## 导出下发数据


In [13]:
output_dir_name = 'output'
out_dir = Path.cwd() / output_dir_name
if not out_dir.exists():
    out_dir.mkdir()

out_filename = f'{tp}-租赁平台-合同规范性检查（下发）.xlsx'

out_path = out_dir / out_filename

with pd.ExcelWriter(out_path) as writer:
    analyze(tp_db).reset_index().to_excel(writer, sheet_name='全量统计')
    analyze(incremental_db).reset_index().to_excel(writer, sheet_name='增量统计')
    tp_db.reset_index().to_excel(writer, sheet_name='全量数据')
    incremental_db.reset_index().to_excel(writer, sheet_name='增量数据')
